In [ ]:
pip install sparse_dot_topn

In [ ]:
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct  # Leading Juice for us
import time
pd.set_option('display.max_colwidth', -1)

#email splitting function definition
def email_split(x):
  token = x.split('@')
  return token[0]


user =  pd.read_csv('/content/users from Aug.csv')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
user =  pd.read_csv('/content/users from Aug.csv')

# Date settings - based on optimised dates we are considering last 14 days user for comparison

user['created'] = pd.to_datetime(user['created'],format = '%Y-%m-%d %H:%M')
user['created'] = user['created'].dt.date
user['days_from_register'] = (user.loc[0,'created']- user['created'] ).apply(lambda x:x.days)
user = user[user['days_from_register']<14]

df = user[['email','username']].drop(columns = ['username'])
df = df.rename(columns={'email':'RoomTypes'})


#email splitting
df['RoomTypes'] = df['RoomTypes'].apply(email_split)
df.head()

,RoomTypes
0,nepalmondal361
1,vishum333
2,notifyamam123
3,dhamgayeindu
4,abhishekyad969


In [ ]:
df.shape
number = df.shape[0]
print(number)

# this number is the amount of users that will be compared. 
# On average the system will have to check 500 - 1000 per each user registration

631


In [ ]:
#  ngrams(here we are taking n = 3 thus 3-gram (trigrams ) as  most room types only contain two or three words
#  used for cleaning and removing some punctuation (dots, comma’s etc) i.e.((,-./)) from a string 
#  and generate and collect all n-grams of the string.  
 
def ngrams(string, n=3):

    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

# Testing ngrams work for verification 

print('All 3-grams in "Deluxroom":')
ngrams('Deluxroom')

All 3-grams in "Deluxroom":


['Del', 'elu', 'lux', 'uxr', 'xro', 'roo', 'oom']

In [ ]:
room_types = df['RoomTypes']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(room_types)

print(tf_idf_matrix[0])

  (0, 197)	0.33324462218204615
  (0, 1714)	0.31324399716000767
  (0, 655)	0.2880461896340714
  (0, 946)	0.26486203356352384
  (0, 1957)	0.2448614085414854
  (0, 2122)	0.27905270285074657
  (0, 1890)	0.25905207782870815
  (0, 1782)	0.31324399716000767
  (0, 670)	0.299053327872785
  (0, 2178)	0.2372575238114217
  (0, 1100)	0.27905270285074657
  (0, 1977)	0.33324462218204615


In [ ]:
# calculate the similarity between two vectors of TF-IDF values the Cosine Similarity is usually used.
# result matrix in a very sparse terms and Scikit-learn deals with this nicely by returning a sparse CSR matrix.

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [ ]:
#  Top 10 with similarity above 0.71
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.71)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.0073511600494384766


In [ ]:
# unpacks the resulting sparse matrix
def get_matches_df(sparse_matrix, name_vector, top=number):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [ ]:
# store the  matches into new dataframe called matched_df and 
# printing 10 samples
matches_df = get_matches_df(matches, room_types, top=number)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # For removing all exact matches
matches_df.sample(10)

,left_side,right_side,similairity
123,tech+testaccount565,tech+testaccount566,0.919886
118,tech+testaccount566,tech+testaccount564,0.919886
126,tech+testaccount564,tech+testaccount565,0.919886
122,tech+testaccount565,tech+testaccount564,0.919886
130,tech+testaccount563,tech+testaccount565,0.919886
127,tech+testaccount564,tech+testaccount566,0.919886
125,tech+testaccount564,tech+testaccount563,0.919886
131,tech+testaccount563,tech+testaccount566,0.919886
117,tech+testaccount566,tech+testaccount563,0.919886
119,tech+testaccount566,tech+testaccount565,0.919886


In [ ]:
# printing the matches in sorted order
matches_df.sort_values(['similairity'], ascending=False).head(10)

,left_side,right_side,similairity
117,tech+testaccount566,tech+testaccount563,0.919886
118,tech+testaccount566,tech+testaccount564,0.919886
119,tech+testaccount566,tech+testaccount565,0.919886
121,tech+testaccount565,tech+testaccount563,0.919886
122,tech+testaccount565,tech+testaccount564,0.919886
123,tech+testaccount565,tech+testaccount566,0.919886
125,tech+testaccount564,tech+testaccount563,0.919886
126,tech+testaccount564,tech+testaccount565,0.919886
127,tech+testaccount564,tech+testaccount566,0.919886
129,tech+testaccount563,tech+testaccount564,0.919886


In [ ]:
matches_df.shape

(12, 3)

In [ ]:
#intializing step, we just assume at all users are genuine, then among them only mark the fraud ones

user1 = user.iloc[0:number]
for i in user1.index:
  user1.loc[i,'Multiple_Accounts'] = 'False'

user2 = pd.merge(user1,df,left_on= user1.index,right_on=df.index)
user2.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,key_0,Unnamed: 0,_id,email,created,isBlocked,isKycComplete,isFullKycComplete,username,enterraUserId,blockReason,blockedBy,blockedOn,days_from_register,Multiple_Accounts,RoomTypes
0,0,1,600c369da868280027776f6b,nepalmondal361@gmail.com,2021-01-23,False,False,False,Nepal123,155119923,NaN,NaN,NaN,0,False,nepalmondal361
1,1,2,600c34a4a868280027776f69,vishum333@gmail.com,2021-01-23,False,False,False,C0mmander,155119587,NaN,NaN,NaN,0,False,vishum333
2,2,3,600c28daa868280027776f61,notifyamam123@gmail.com,2021-01-23,False,False,False,igotthenuts,155113168,NaN,NaN,NaN,0,False,notifyamam123
3,3,4,600c1cb5a868280027776f59,dhamgayeindu@gmail.com,2021-01-23,False,False,False,shetty5,155107318,NaN,NaN,NaN,0,False,dhamgayeindu
4,4,5,600bfb5ea868280027776f47,abhishekyad969@gmail.com,2021-01-23,False,False,False,Abhi969,155089556,NaN,NaN,NaN,0,False,abhishekyad969


In [ ]:
#cross checkking simialrity with initial database and then flagging the fraud ones

for i in user2.index:
  for j in matches_df.index:
   if  user2.loc[i,'RoomTypes'] == matches_df.loc[j,'left_side']:
     user2.loc[i,'Multiple_Accounts'] = 'True'
   if  user2.loc[i,'RoomTypes'] == matches_df.loc[j,'right_side']:
     user2.loc[i,'Multiple_Accounts'] = 'True'   

user2.Multiple_Accounts.value_counts()

False    627
True     4  
Name: Multiple_Accounts, dtype: int64

In [ ]:
#these are the fraud users registered in last 14 days

user2[user2['Multiple_Accounts']=='True']

,key_0,Unnamed: 0,_id,email,created,isBlocked,isKycComplete,isFullKycComplete,username,enterraUserId,blockReason,blockedBy,blockedOn,days_from_register,Multiple_Accounts,RoomTypes
116,116,117,60069aff27fcd60026cb53a6,tech+testaccount566@9stacks.com,2021-01-19,False,False,False,Testaccount566,154620830,testing,farhan@9stacks.com,NaN,4,True,tech+testaccount566
117,117,118,60069aba27fcd60026cb53a4,tech+testaccount565@9stacks.com,2021-01-19,False,False,False,Testaccount565,154620242,NaN,NaN,NaN,4,True,tech+testaccount565
118,118,119,60069a7427fcd60026cb53a2,tech+testaccount564@9stacks.com,2021-01-19,False,False,False,Testaccount564,154620181,NaN,NaN,NaN,4,True,tech+testaccount564
119,119,120,60069a1a27fcd60026cb53a0,tech+testaccount563@9stacks.com,2021-01-19,False,False,False,Testaccount563,154620123,NaN,NaN,NaN,4,True,tech+testaccount563


Now for matching username too

In [ ]:
# reading dataset as df
df =  pd.read_csv('/content/room_type (1).csv')


df = df.iloc[0:number]

In [ ]:
#  ngrams(here we are taking n = 3 thus 3-gram (trigrams ) as  most room types only contain two or three words
#  used for cleaning and removing some punctuation (dots, comma’s etc) i.e.((,-./)) from a string 
#  and generate and collect all n-grams of the string.  

 
def ngrams(string, n=3):

    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


# Testing ngrams work for verification 

print('All 3-grams in "Deluxroom":')
ngrams('Deluxroom')


room_types = df['RoomTypes']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(room_types)

print(tf_idf_matrix[0])

All 3-grams in "Deluxroom":
  (0, 34)	0.3019252320906027
  (0, 1601)	0.4580411111813528
  (0, 802)	0.43055046947367814
  (0, 1955)	0.383554907067911
  (0, 1144)	0.39591635690213695
  (0, 521)	0.4580411111813528


In [ ]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [ ]:
#  Top 10 with similarity above 0.71
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.71)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.0019102096557617188


In [ ]:
# unpacks the resulting sparse matrix
def get_matches_df(sparse_matrix, name_vector, top=number):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [ ]:
# store the  matches into new dataframe called matched_df and 
# printing 10 samples
matches_df = get_matches_df(matches, room_types, top=number)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # For removing all exact matches
matches_df.sample(10)

,left_side,right_side,similairity
547,Vibia7,Hibia7,0.755223
543,Vibia9,Hibia9,0.755223
460,Ankit1498,Ankit1497,0.726328
131,Testaccount563,Testaccount566,0.891527
290,Ankit1518,Ankit1511,0.723275
287,Ankit1519,Ankit1511,0.723275
304,Ankit1512,Ankit1511,0.723275
466,Ankit1496,Ankit1497,0.726328
458,Ankit1498,Ankit1494,0.726328
472,Ankit1497,Ankit1496,0.726328


In [ ]:
matches_df.shape

(131, 3)

In [ ]:
# printing the matches in sorted order
kv = matches_df.sort_values(['similairity'], ascending=False)
matches_df.sort_values(['similairity'], ascending=False).head(10)

,left_side,right_side,similairity
121,Testaccount565,Testaccount563,0.897526
129,Testaccount563,Testaccount565,0.897526
117,Testaccount566,Testaccount563,0.891527
130,Testaccount563,Testaccount564,0.891527
118,Testaccount566,Testaccount565,0.891527
122,Testaccount565,Testaccount564,0.891527
123,Testaccount565,Testaccount566,0.891527
125,Testaccount564,Testaccount563,0.891527
126,Testaccount564,Testaccount565,0.891527
131,Testaccount563,Testaccount566,0.891527


In [ ]:
kv1 = kv.left_side.unique()
kv1.shape

(59,)

In [ ]:
matches_df.shape

(131, 3)

In [ ]:
kv.shape

(131, 3)

In [ ]:
#cross checkking simialrity with initial database and then flagging the fraud ones

for k in user2.index:
  for l in kv.index:
   if  user2.loc[k,'username'] == kv.loc[l,'left_side']:
     user2.loc[k,'Multiple_Accounts'] = 'True'
   if  user2.loc[k,'username'] == kv.loc[l,'right_side']:
     user2.loc[k,'Multiple_Accounts'] = 'True'    

user2.head()

,key_0,Unnamed: 0,_id,email,created,isBlocked,isKycComplete,isFullKycComplete,username,enterraUserId,blockReason,blockedBy,blockedOn,days_from_register,Multiple_Accounts,RoomTypes
0,0,1,600c369da868280027776f6b,nepalmondal361@gmail.com,2021-01-23,False,False,False,Nepal123,155119923,NaN,NaN,NaN,0,False,nepalmondal361
1,1,2,600c34a4a868280027776f69,vishum333@gmail.com,2021-01-23,False,False,False,C0mmander,155119587,NaN,NaN,NaN,0,False,vishum333
2,2,3,600c28daa868280027776f61,notifyamam123@gmail.com,2021-01-23,False,False,False,igotthenuts,155113168,NaN,NaN,NaN,0,False,notifyamam123
3,3,4,600c1cb5a868280027776f59,dhamgayeindu@gmail.com,2021-01-23,False,False,False,shetty5,155107318,NaN,NaN,NaN,0,False,dhamgayeindu
4,4,5,600bfb5ea868280027776f47,abhishekyad969@gmail.com,2021-01-23,False,False,False,Abhi969,155089556,NaN,NaN,NaN,0,False,abhishekyad969


In [ ]:
user2.Multiple_Accounts.value_counts()

False    571
True     60 
Name: Multiple_Accounts, dtype: int64

In [ ]:
#these are the fraud users registered in last 14 days

user2[user2['Multiple_Accounts']=='True']

,key_0,Unnamed: 0,_id,email,created,isBlocked,isKycComplete,isFullKycComplete,username,enterraUserId,blockReason,blockedBy,blockedOn,days_from_register,Multiple_Accounts,RoomTypes
116,116,117,60069aff27fcd60026cb53a6,tech+testaccount566@9stacks.com,2021-01-19,False,False,False,Testaccount566,154620830,testing,farhan@9stacks.com,NaN,4,True,tech+testaccount566
117,117,118,60069aba27fcd60026cb53a4,tech+testaccount565@9stacks.com,2021-01-19,False,False,False,Testaccount565,154620242,NaN,NaN,NaN,4,True,tech+testaccount565
118,118,119,60069a7427fcd60026cb53a2,tech+testaccount564@9stacks.com,2021-01-19,False,False,False,Testaccount564,154620181,NaN,NaN,NaN,4,True,tech+testaccount564
119,119,120,60069a1a27fcd60026cb53a0,tech+testaccount563@9stacks.com,2021-01-19,False,False,False,Testaccount563,154620123,NaN,NaN,NaN,4,True,tech+testaccount563
204,204,205,6002783a27fcd60026cb50b1,simba.sa.h.u988@gmail.com,2021-01-16,False,False,False,Kuma10,154274907,NaN,NaN,NaN,7,True,simba.sa.h.u988
214,214,215,6002743a27fcd60026cb50a2,simba.sah.u9.8.8@gmail.com,2021-01-16,True,False,False,Kuma1,154274583,Chip dumping,ajaj@9stacks.com,1.610806e+12,7,True,simba.sah.u9.8.8
243,243,244,6001349b27fcd60026cb4fc9,yashphadke0607@gmail.com,2021-01-15,False,False,False,Yashsureshphadke,154162004,NaN,NaN,NaN,8,True,yashphadke0607
255,255,256,6000ed4327fcd60026cb4f5d,durgeshsingh53555@gmail.com,2021-01-15,True,False,False,Bhanu2999,154148634,Multiple account,pradeep.mondal@9stacks.com,1.610689e+12,8,True,durgeshsingh53555
270,270,271,60008ad327fcd60026cb4f2b,r.a.mku.marz.z.z1002@gmail.com,2021-01-14,True,False,False,Ankit1519,154130568,Multiple account,pradeep.mondal@9stacks.com,1.610689e+12,9,True,r.a.mku.marz.z.z1002
272,272,273,60008a7527fcd60026cb4f29,r.am.k.u.marz.zz1.002@gmail.com,2021-01-14,True,False,False,Ankit1518,154130499,Multiple account,pradeep.mondal@9stacks.com,1.610689e+12,9,True,r.am.k.u.marz.zz1.002
